# Proyek Akhir Pencarian Dan Penambangan WEB (Klasifikasi Berita dengan Reduksi Dimensi Latent Dirichlet Allocation (LDA) menggunakan Metode Naive Bayes)

## Deployments

[Streamlit Deployments](https://app-project-ppw.streamlit.app/)

## Import Library

Kode di bawah adalah implementasi dalam bahasa Python untuk pemrosesan teks dan pembelajaran mesin menggunakan beberapa pustaka. Berikut adalah penjelasan singkat untuk setiap bagian kode tersebut:

1. **Import Libraries/Modules:**
   Import pustaka-pustaka yang dibutuhkan untuk pemrosesan teks dan pembelajaran mesin, termasuk scikit-learn (sklearn) untuk metrik evaluasi dan algoritma pembelajaran mesin, NLTK untuk pemrosesan bahasa alami, Pandas untuk manipulasi data, dan lain-lain.

2. **Download Stopwords dan Punkt dari NLTK:**
   Mengunduh dataset stopwords dan Punkt tokenizer dari NLTK. Stopwords digunakan untuk menghilangkan kata-kata umum yang tidak memberikan banyak informasi, dan Punkt digunakan untuk tokenisasi teks.

3. **Filter Warnings:**
   Menonaktifkan peringatan yang mungkin muncul selama eksekusi program untuk menghindari gangguan.



In [1]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.svm import SVC

import pandas as pd
import warnings
import joblib
import nltk
import re

nltk.download('stopwords')
nltk.download('punkt')
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Load Dataset


Kode di bawah menggunakan pustaka Pandas untuk membaca data dari file CSV yang berisi berita. Fungsi `pd.read_csv` digunakan untuk membaca file CSV yang berlokasi di '/content/drive/MyDrive/PPW/PPW/Resource/Data Berita CNN 1000 data.csv'. Hasilnya disimpan dalam variabel `df`, yang merupakan DataFrame Pandas. DataFrame adalah struktur data tabular yang memungkinkan manipulasi dan analisis data. Akhirnya, kode tersebut menampilkan isi DataFrame `df`.

In [2]:
df = pd.read_csv('/content/drive/MyDrive/PPW/PPW/Resource/Data Berita CNN 1000 data.csv')
df

,Url,Judul,Tanggal,Artikel,Category
0,https://www.cnnindonesia.com/teknologi/2023121...,Siapa yang Pertama Menyebar Narasi Kebencian s...,"Rabu, 13 Des 2023 17:20 WIB","Isu negatif, hoaks, hingga ujaran kebencian me...",Teknologi
1,https://www.cnnindonesia.com/teknologi/2023121...,COP28 Serukan Peralihan Penggunaan Bahan Bakar...,"Rabu, 13 Des 2023 15:51 WIB","Rancangan kesepakatan iklim PBB,COP28 pada har...",Teknologi
2,https://www.cnnindonesia.com/teknologi/2023121...,"Rekor TikTok, Pengguna Habiskan Uang Rp156 Tri...","Rabu, 13 Des 2023 14:21 WIB",Platform media sosial TikTokmemecahkan rekor s...,Teknologi
3,https://www.cnnindonesia.com/teknologi/2023121...,"Kemenangan Epic, Google Tumbang di Sidang Anti...","Rabu, 13 Des 2023 12:51 WIB","Epic Games, pembuat gim 'Fortnite' menang di p...",Teknologi
4,https://www.cnnindonesia.com/teknologi/2023121...,"Mengenal JAKI, Aplikasi yang Dipamerkan Anies ...","Rabu, 13 Des 2023 10:05 WIB",Aplikasi JAKImendapat sorotan warganet usai ca...,Teknologi
...,...,...,...,...,...
1495,https://www.cnnindonesia.com/olahraga/20231127...,Fabio Di Giannantonio Resmi Gabung VR46 di Mot...,"Senin, 27 Nov 2023 18:24 WIB",TimVR46 resmi mengumumkan perekrutan Fabio Di ...,Olahraga
1496,https://www.cnnindonesia.com/olahraga/20231127...,Bezzecchi Diklaim Lontarkan Kata-kata Kasar ke...,"Senin, 27 Nov 2023 16:44 WIB",Marc Marquezmenyebut Marco Bezzecchi melontark...,Olahraga
1497,https://www.cnnindonesia.com/olahraga/20231127...,Jadwal Prancis vs Mali di Semifinal Piala Duni...,"Senin, 27 Nov 2023 16:17 WIB",DuelPrancis U-17 vsMali akan terjadi dalam sem...,Olahraga
1498,https://www.cnnindonesia.com/olahraga/20231127...,Jadwal Argentina vs Jerman di Semifinal Piala ...,"Senin, 27 Nov 2023 15:03 WIB",Timnas Argentina U-17akan melawanJerman pada b...,Olahraga


## Cek NULL Data

Kode `df.isnull().sum()` digunakan untuk menghitung jumlah nilai null di setiap kolom dalam DataFrame `df`. Hasilnya adalah Series Pandas yang menunjukkan berapa banyak nilai null yang terdapat pada masing-masing kolom data. Pengecekan ini membantu identifikasi dan penanganan awal terhadap nilai null dalam dataset.

In [3]:
df.isnull().sum()

Url         0
Judul       0
Tanggal     0
Artikel     0
Category    0
dtype: int64

## Cleaning

Kode di bawah mendefinisikan fungsi `cleaning` yang bertujuan membersihkan teks dari karakter non-alfabet dan spasi yang tidak diinginkan. Proses pembersihan ini kemudian diterapkan pada kolom 'Artikel' dalam DataFrame `df`, dan hasilnya disimpan dalam kolom baru 'Cleaning'. Hasil akhir berupa DataFrame yang berisi teks yang telah dibersihkan.

In [4]:
def cleaning(text):
  text = re.sub(r'[^a-zA-Z\s]', '', text).strip()
  return text

df['Cleaning'] = df['Artikel'].apply(cleaning)
df['Cleaning']

0       Isu negatif hoaks hingga ujaran kebencian meng...
1       Rancangan kesepakatan iklim PBBCOP pada hari R...
2       Platform media sosial TikTokmemecahkan rekor s...
3       Epic Games pembuat gim Fortnite menang di peng...
4       Aplikasi JAKImendapat sorotan warganet usai ca...
                              ...                        
1495    TimVR resmi mengumumkan perekrutan Fabio Di Gi...
1496    Marc Marquezmenyebut Marco Bezzecchi melontark...
1497    DuelPrancis U vsMali akan terjadi dalam semifi...
1498    Timnas Argentina Uakan melawanJerman pada baba...
1499    Pembalap VR Marco Bezzecchi melabrakMarc Marqu...
Name: Cleaning, Length: 1500, dtype: object

## Tokenizing

Kode di bawah mendefinisikan fungsi `tokenizer` yang mengonversi teks menjadi huruf kecil dan kemudian melakukan tokenisasi kata. Proses tokenisasi ini diterapkan pada kolom 'Cleaning' dalam DataFrame `df`, dan hasilnya disimpan dalam kolom baru 'Tokenizing'. Hasil akhirnya berupa DataFrame yang berisi daftar kata-kata yang telah dihasilkan dari tokenisasi teks.

In [5]:
def tokenizer(text):
  text = text.lower()
  return word_tokenize(text)

df['Tokenizing'] = df['Cleaning'].apply(tokenizer)
df['Tokenizing']

0       [isu, negatif, hoaks, hingga, ujaran, kebencia...
1       [rancangan, kesepakatan, iklim, pbbcop, pada, ...
2       [platform, media, sosial, tiktokmemecahkan, re...
3       [epic, games, pembuat, gim, fortnite, menang, ...
4       [aplikasi, jakimendapat, sorotan, warganet, us...
                              ...                        
1495    [timvr, resmi, mengumumkan, perekrutan, fabio,...
1496    [marc, marquezmenyebut, marco, bezzecchi, melo...
1497    [duelprancis, u, vsmali, akan, terjadi, dalam,...
1498    [timnas, argentina, uakan, melawanjerman, pada...
1499    [pembalap, vr, marco, bezzecchi, melabrakmarc,...
Name: Tokenizing, Length: 1500, dtype: object

## Stopword Removal

Kode di bawah melakukan penghapusan stopwords dari setiap baris teks dalam kolom 'Tokenizing' DataFrame `df` yang telah melibatkan beberapa langkah:

1. **Pembuatan Daftar Stopwords (corpus):**
   Membuat daftar stopwords dalam bahasa Indonesia menggunakan modul `stopwords` dari NLTK.

2. **Fungsi Stopword Removal:**
   Fungsi `stopwordText` dibuat untuk menghapus stopwords dari daftar kata-kata dan mengembalikan daftar kata tanpa stopwords.

3. **Penerapan Stopword Removal pada DataFrame:**
   Mengaplikasikan fungsi `stopwordText` pada setiap baris dalam kolom 'Tokenizing', dan hasilnya disimpan dalam kolom baru 'Stopword Removal'.

4. **Gabungan Token menjadi Kalimat Utuh:**
   Menggabungkan kembali token-token yang telah dihasilkan setelah penghapusan stopwords menjadi satu kalimat utuh. Hasilnya disimpan dalam kolom baru 'Full Text'.

5. **Menampilkan DataFrame Hasil Akhir:**
   Menampilkan kolom 'Full Text' dari DataFrame, yang berisi teks yang telah melalui proses penghapusan stopwords dan penggabungan token.

In [6]:
corpus = stopwords.words('indonesian')

def stopwordText(words):
 return [word for word in words if word not in corpus]

df['Stopword Removal'] = df['Tokenizing'].apply(stopwordText)

# Gabungkan kembali token menjadi kalimat utuh
df['Full Text'] = df['Stopword Removal'].apply(lambda x: ' '.join(x))
df['Full Text']

0       isu negatif hoaks ujaran kebencian pengungsi r...
1       rancangan kesepakatan iklim pbbcop rabu menyer...
2       platform media sosial tiktokmemecahkan rekor s...
3       epic games pembuat gim fortnite menang pengadi...
4       aplikasi jakimendapat sorotan warganet calon p...
                              ...                        
1495    timvr resmi mengumumkan perekrutan fabio giann...
1496    marc marquezmenyebut marco bezzecchi melontark...
1497    duelprancis u vsmali semifinal piala dunia u s...
1498    timnas argentina uakan melawanjerman babak sem...
1499    pembalap vr marco bezzecchi melabrakmarc marqu...
Name: Full Text, Length: 1500, dtype: object

## TFIDF

Kode di bawah mendefinisikan fungsi `tfidf` untuk menghitung skor TF-IDF dari teks dokumen menggunakan TfidfVectorizer dari scikit-learn. Fungsi ini kemudian diterapkan pada DataFrame `df`, menghasilkan matriks TF-IDF dan DataFrame akhir yang berisi skor TF-IDF untuk setiap kata dalam setiap dokumen, bersama dengan teks dokumen asli dan kategorinya. Proses ini membantu mengubah data teks menjadi representasi numerik yang dapat digunakan dalam model pembelajaran mesin.

In [7]:
def tfidf(dokumen, category):
  vectorizer = TfidfVectorizer()
  x = vectorizer.fit_transform(dokumen).toarray()
  terms = vectorizer.get_feature_names_out()

  final_tfidf = pd.DataFrame(x, columns=terms)
  final_tfidf.insert(0, 'Dokumen', dokumen)
  final_tfidf.insert(len(final_tfidf.columns),'Category', category)

  return (vectorizer, final_tfidf)

tfidf_vectorizer, final_tfidf = tfidf(df['Full Text'], df['Category'])
final_tfidf

,Dokumen,aaa,aal,aali,aalx,aaron,aaui,abad,abadabad,abadi,...,zulhendrizal,zulikflihasanuntuk,zulkifil,zulkifilmengatakan,zulkifli,zulkifliberharap,zulkiflimengimbaupemerintah,zulkiflimenyebut,zurich,Category
0,isu negatif hoaks ujaran kebencian pengungsi r...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Teknologi
1,rancangan kesepakatan iklim pbbcop rabu menyer...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Teknologi
2,platform media sosial tiktokmemecahkan rekor s...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Teknologi
3,epic games pembuat gim fortnite menang pengadi...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Teknologi
4,aplikasi jakimendapat sorotan warganet calon p...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Teknologi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,timvr resmi mengumumkan perekrutan fabio giann...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Olahraga
1496,marc marquezmenyebut marco bezzecchi melontark...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Olahraga
1497,duelprancis u vsmali semifinal piala dunia u s...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Olahraga
1498,timnas argentina uakan melawanjerman babak sem...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Olahraga


## Deklarasi X dan y dengan drop fitur dokumen dan label

Kode dibawah membuat variabel `X` yang berisi matriks fitur untuk digunakan dalam model pembelajaran mesin. Matriks fitur ini diperoleh dari DataFrame `final_tfidf` dengan menghapus dua kolom, yaitu 'Dokumen' dan 'Category'. Variabel `X` dapat digunakan sebagai input untuk melatih model machine learning, di mana setiap baris mewakili dokumen dan setiap kolom mewakili fitur dengan skor TF-IDF.

In [8]:
X = final_tfidf.drop(['Dokumen', 'Category'], axis=1)
X

,aaa,aal,aali,aalx,aaron,aaui,abad,abadabad,abadi,abadipt,...,zulhassukses,zulhendrizal,zulikflihasanuntuk,zulkifil,zulkifilmengatakan,zulkifli,zulkifliberharap,zulkiflimengimbaupemerintah,zulkiflimenyebut,zurich
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Kode dibawah membuat variabel `y` yang berisi target atau label dari setiap dokumen dalam DataFrame `df`. Variabel `y` ini menyimpan informasi kategori atau kelas dari masing-masing dokumen. Sebagai contoh, jika dokumen pertama memiliki label 'Politik', maka nilai `y` untuk dokumen tersebut akan 'Politik'. Variabel `y` ini biasanya digunakan sebagai target yang ingin diprediksi dalam model pembelajaran mesin.

In [9]:
y = df['Category']
y

0       Teknologi
1       Teknologi
2       Teknologi
3       Teknologi
4       Teknologi
          ...    
1495     Olahraga
1496     Olahraga
1497     Olahraga
1498     Olahraga
1499     Olahraga
Name: Category, Length: 1500, dtype: object

## Splitting Data

Kode di bawah menggunakan fungsi `train_test_split` dari scikit-learn untuk membagi data menjadi dua set: set pelatihan (`X_train`, `y_train`) dan set pengujian (`X_test`, `y_test`). Proporsi data yang dialokasikan untuk set pengujian adalah 30%, dan seed (random_state) sebesar 42 digunakan untuk memastikan reproduktibilitas hasil pembagian data. Set pelatihan digunakan untuk melatih model, sedangkan set pengujian digunakan untuk menguji seberapa baik model dapat menggeneralisasi ke data yang tidak dilihat sebelumnya.

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## **Modeling With LDA**

### Mencari Best Parameter untuk LDA

Kode dibawah mendefinisikan fungsi `find_best_lda` untuk mencari kombinasi parameter terbaik dalam model Latent Dirichlet Allocation (LDA) dan model Naive Bayes. Fungsi ini melakukan eksplorasi sejumlah kombinasi parameter (jumlah topik, alpha, dan beta), melatih model, dan menyimpan parameter terbaik beserta model dan hasil pelatihan. Selanjutnya, fungsi ini memanggil dengan matriks fitur pelatihan dan pengujian (`X_train` dan `X_test`), serta daftar nilai untuk jumlah topik, alpha, dan beta. Hasilnya disimpan dalam variabel `best_param` untuk parameter terbaik dan `history` untuk mencatat hasil eksperimen. Proses ini membantu dalam menemukan konfigurasi parameter yang memberikan akurasi terbaik pada model.

In [11]:
def find_best_lda(Xtrain, Xtest, n_components, alpha, beta):
  looping = 1
  best = {'k' : 0, 'alpha' : 0, 'beta' : 0, 'accuracy' : 0, 'model': '', 'lda' : '', 'lda_Xtrain' : '', 'lda_Xtest' : ''}
  history = pd.DataFrame(columns=["Pengujian Ke", "K", "Alpha", "Beta", "Accuracy"])

  # Menambahkan tqdm pada loop terluar
  for k in n_components:
    for a in alpha:
      for b in beta:
        lda = LatentDirichletAllocation(n_components=k, doc_topic_prior=a, topic_word_prior=b)
        lda_Xtrain = lda.fit_transform(Xtrain)
        lda_Xtest = lda.transform(Xtest)

        # Membuat model Naive Bayes
        model = GaussianNB()

        # Melatih model pada data pelatihan
        model.fit(lda_Xtrain, y_train)

        # Melakukan prediksi pada data pengujian
        y_pred = model.predict(lda_Xtest)

        # Menghitung akurasi
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Jumlah Topik: {k}, Alpha: {a}, Beta: {b}, Accuracy: {accuracy}")

        if accuracy > best['accuracy']:
          best['accuracy'] = accuracy
          best['k'] = k
          best['alpha'] = a
          best['beta'] = b
          best['model'] = model
          best['lda'] = lda
          best['lda_Xtrain'] = lda_Xtrain
          best['lda_Xtest'] = lda_Xtest

        history.loc[len(history)] = [f"Pengujian Ke- {looping}", k, a, b, accuracy]
        looping += 1

  return (best, history)

k = [3, 4, 5]
alpha = [0.3, 0.4]
beta = [0.1, 0.2]
best_param, history = find_best_lda(X_train, X_test, k, alpha, beta)

Jumlah Topik: 3, Alpha: 0.3, Beta: 0.1, Accuracy: 0.5022222222222222
Jumlah Topik: 3, Alpha: 0.3, Beta: 0.2, Accuracy: 0.4577777777777778
Jumlah Topik: 3, Alpha: 0.4, Beta: 0.1, Accuracy: 0.5555555555555556
Jumlah Topik: 3, Alpha: 0.4, Beta: 0.2, Accuracy: 0.5377777777777778
Jumlah Topik: 4, Alpha: 0.3, Beta: 0.1, Accuracy: 0.6933333333333334
Jumlah Topik: 4, Alpha: 0.3, Beta: 0.2, Accuracy: 0.4955555555555556
Jumlah Topik: 4, Alpha: 0.4, Beta: 0.1, Accuracy: 0.6244444444444445
Jumlah Topik: 4, Alpha: 0.4, Beta: 0.2, Accuracy: 0.7288888888888889
Jumlah Topik: 5, Alpha: 0.3, Beta: 0.1, Accuracy: 0.5644444444444444
Jumlah Topik: 5, Alpha: 0.3, Beta: 0.2, Accuracy: 0.6
Jumlah Topik: 5, Alpha: 0.4, Beta: 0.1, Accuracy: 0.5222222222222223
Jumlah Topik: 5, Alpha: 0.4, Beta: 0.2, Accuracy: 0.7288888888888889


Variabel `best_param` berisi hasil terbaik dari eksperimen untuk mencari parameter optimal pada model Latent Dirichlet Allocation (LDA) dan model Naive Bayes. Informasi ini dicatat dari hasil pelatihan dan pengujian sejumlah kombinasi parameter (jumlah topik, alpha, dan beta) pada dataset.

Struktur `best_param` adalah sebuah dictionary dengan beberapa kunci (keys) yang menyimpan informasi terbaik sebagai berikut:

- `'k'`: Jumlah topik terbaik yang ditemukan.
- `'alpha'`: Nilai alpha terbaik.
- `'beta'`: Nilai beta terbaik.
- `'accuracy'`: Akurasi terbaik yang dicapai pada model.
- `'model'`: Model Naive Bayes terbaik yang telah dilatih.
- `'lda'`: Model LDA terbaik.
- `'lda_Xtrain'`: Representasi dokumen dalam ruang topik dari data pelatihan.
- `'lda_Xtest'`: Representasi dokumen dalam ruang topik dari data pengujian.

Variabel `best_param` ini menyediakan hasil optimal dari eksperimen dan dapat digunakan untuk mengevaluasi performa model terpilih pada data pengujian.

In [12]:
best_param

{'k': 4,
 'alpha': 0.4,
 'beta': 0.2,
 'accuracy': 0.7288888888888889,
 'model': GaussianNB(),
 'lda': LatentDirichletAllocation(doc_topic_prior=0.4, n_components=4,
                           topic_word_prior=0.2),
 'lda_Xtrain': array([[0.28276356, 0.57711118, 0.03252148, 0.10760378],
        [0.05128478, 0.04520822, 0.0426631 , 0.8608439 ],
        [0.64823988, 0.04753441, 0.1143168 , 0.18990892],
        ...,
        [0.04703812, 0.85972139, 0.04578296, 0.04745753],
        [0.05108968, 0.05137631, 0.05053068, 0.84700332],
        [0.05243489, 0.05118098, 0.05132394, 0.8450602 ]]),
 'lda_Xtest': array([[0.05402348, 0.04586847, 0.04570473, 0.85440332],
        [0.36905552, 0.07510736, 0.0512546 , 0.50458251],
        [0.63132314, 0.12768622, 0.04248796, 0.19850269],
        ...,
        [0.39312557, 0.07113671, 0.04673263, 0.48900509],
        [0.13597252, 0.07628387, 0.04939392, 0.73834969],
        [0.05093968, 0.04850218, 0.04640635, 0.85415179]])}

Variabel `history` adalah sebuah DataFrame Pandas yang merekam hasil eksperimen untuk mencari parameter terbaik pada model Latent Dirichlet Allocation (LDA) dan model Naive Bayes. DataFrame ini menyimpan sejarah akurasi dari berbagai percobaan dengan kombinasi parameter yang berbeda, seperti jumlah topik, alpha, dan beta. Masing-masing baris dalam DataFrame merepresentasikan satu percobaan dengan informasi tentang nomor eksperimen, jumlah topik, nilai alpha, nilai beta, dan akurasi yang dicapai pada model. `history` dapat digunakan untuk menganalisis perubahan performa model seiring variasi parameter yang diuji.

In [13]:
history

,Pengujian Ke,K,Alpha,Beta,Accuracy
0,Pengujian Ke- 1,3,0.3,0.1,0.502222
1,Pengujian Ke- 2,3,0.3,0.2,0.457778
2,Pengujian Ke- 3,3,0.4,0.1,0.555556
3,Pengujian Ke- 4,3,0.4,0.2,0.537778
4,Pengujian Ke- 5,4,0.3,0.1,0.693333
5,Pengujian Ke- 6,4,0.3,0.2,0.495556
6,Pengujian Ke- 7,4,0.4,0.1,0.624444
7,Pengujian Ke- 8,4,0.4,0.2,0.728889
8,Pengujian Ke- 9,5,0.3,0.1,0.564444
9,Pengujian Ke- 10,5,0.3,0.2,0.600000


Kode `history.to_csv("history.csv", index=False)` digunakan untuk menyimpan DataFrame `history` dalam bentuk file CSV. Berikut penjelasan singkatnya:

- **`history.to_csv("history.csv", index=False)`:**
  - `history.to_csv`: Metode Pandas untuk menyimpan DataFrame ke dalam file CSV.
  - `"history.csv"`: Nama file CSV yang akan dibuat atau ditimpa.
  - `index=False`: Parameter yang mengindikasikan untuk tidak menyimpan indeks baris DataFrame ke dalam file CSV.

Dengan menjalankan kode ini, DataFrame `history` akan disimpan sebagai file CSV dengan nama "history.csv" tanpa menyertakan indeks baris dalam file tersebut. Ini memungkinkan penggunaan file CSV untuk analisis atau visualisasi di luar lingkungan Python atau dalam proyek lainnya.

In [14]:
history.to_csv("history.csv", index=False)

### Deklarasi K, Alpha, dan Beta

### LDA

Kode di bawah mengambil informasi terbaik dari hasil eksperimen yang disimpan dalam variabel `best_param` untuk digunakan lebih lanjut. Berikut penjelasan singkatnya:

- **`lda = best_param['lda']`:**
  - Mengambil model Latent Dirichlet Allocation (LDA) terbaik dari `best_param` dan menyimpannya dalam variabel `lda`. Model ini memiliki konfigurasi parameter terbaik yang ditemukan selama eksperimen.

- **`lda_x_train = best_param['lda_Xtrain']`:**
  - Mengambil representasi dokumen dalam ruang topik dari data pelatihan (hasil transformasi LDA) dan menyimpannya dalam variabel `lda_x_train`. Representasi ini digunakan sebagai matriks fitur untuk melatih model machine learning.

- **`lda_x_test = best_param['lda_Xtest']`:**
  - Mengambil representasi dokumen dalam ruang topik dari data pengujian (hasil transformasi LDA) dan menyimpannya dalam variabel `lda_x_test`. Representasi ini digunakan sebagai matriks fitur untuk menguji model machine learning.

Dengan menyimpan hasil terbaik dari eksperimen pada `best_param`, kita dapat menggunakan model LDA yang optimal dan representasi dokumen dalam ruang topik yang dihasilkan pada tahap selanjutnya dalam analisis atau pemodelan data.

In [15]:
lda = best_param['lda']
lda_x_train = best_param['lda_Xtrain']
lda_x_test = best_param['lda_Xtest']

### Tampilan Hasil Reduksi Dimensi

Kode di bawah membuat DataFrame baru (`output_proporsi_TD`) yang menunjukkan proporsi setiap topik dalam setiap dokumen berdasarkan hasil transformasi model Latent Dirichlet Allocation (LDA) pada data pelatihan. Proses ini melibatkan pembuatan nama kolom untuk masing-masing topik, penggabungan hasil transformasi LDA dengan teks dokumen asli, dan penambahan informasi kategori dokumen. Hasil akhirnya memberikan representasi yang lebih terstruktur tentang bagaimana setiap dokumen terdiri dari berbagai topik menurut model LDA yang optimal. DataFrame ini dapat digunakan untuk analisis lebih lanjut atau sebagai input untuk tahap-tahap selanjutnya dalam pemrosesan atau pemodelan data.

In [16]:
topik_columns = [f"Topik {i}" for i in range(1, best_param['k']+1)]
dokumen = final_tfidf['Dokumen']
output_proporsi_TD = pd.DataFrame(lda_x_train, columns=topik_columns)
output_proporsi_TD.insert(0,'Dokumen', dokumen)
output_proporsi_TD.insert(len(output_proporsi_TD.columns),'Category', final_tfidf['Category'])
output_proporsi_TD

,Dokumen,Topik 1,Topik 2,Topik 3,Topik 4,Category
0,isu negatif hoaks ujaran kebencian pengungsi r...,0.282764,0.577111,0.032521,0.107604,Teknologi
1,rancangan kesepakatan iklim pbbcop rabu menyer...,0.051285,0.045208,0.042663,0.860844,Teknologi
2,platform media sosial tiktokmemecahkan rekor s...,0.648240,0.047534,0.114317,0.189909,Teknologi
3,epic games pembuat gim fortnite menang pengadi...,0.773926,0.050074,0.043751,0.132250,Teknologi
4,aplikasi jakimendapat sorotan warganet calon p...,0.105102,0.769604,0.043250,0.082044,Teknologi
...,...,...,...,...,...,...
1045,kejadian patut ditiru lanjutan liga sepak bola...,0.070571,0.048774,0.045304,0.835351,Olahraga
1046,manchester unitedakan menjamu bayern munchen l...,0.132950,0.528123,0.041669,0.297257,Olahraga
1047,wasit halil umut melerasal turki sorotan korba...,0.047038,0.859721,0.045783,0.047458,Olahraga
1048,tragedi pemukulan wasit halil umut melerdi lig...,0.051090,0.051376,0.050531,0.847003,Olahraga


### Save Data hasil reduksi dimensi

In [17]:
output_proporsi_TD.to_csv('reduksi dimensi.csv', index=False)

### Tampilan proporsi kata di tiap topik

Kode di bawah membuat DataFrame baru (`distribusi_kata_topik`) yang menyajikan informasi tentang distribusi kata-kata pada masing-masing topik dalam model Latent Dirichlet Allocation (LDA). Setiap baris dalam DataFrame ini mewakili satu topik, dan setiap kolom mencerminkan bobot (weight) kata-kata dalam membentuk topik tersebut. DataFrame ini memberikan pandangan tentang kontribusi relatif setiap kata terhadap pembentukan setiap topik dalam model LDA. Data ini dapat digunakan untuk mengeksplorasi makna dan ciri khas masing-masing topik.

In [18]:
# Output distribusi kata pada topik
distribusi_kata_topik = pd.DataFrame(lda.components_)
distribusi_kata_topik

,0,1,2,3,4,5,6,7,8,9,...,28067,28068,28069,28070,28071,28072,28073,28074,28075,28076
0,0.2,0.2,0.200384,0.219586,0.264621,0.202933,0.426473,0.265913,0.331818,0.229393,...,0.2,0.2,0.213526,0.210152,0.2,0.646073,0.2,0.200755,0.200755,0.2
1,0.2,0.2,0.200296,0.208832,0.200428,0.200999,0.200697,0.200321,0.308944,0.201200,...,0.2,0.2,0.291196,0.292300,0.2,1.065649,0.2,0.275876,0.275876,0.2
2,0.2,0.2,0.200306,0.200366,0.200436,0.200278,0.200431,0.200377,0.216678,0.200205,...,0.2,0.2,0.200389,0.200818,0.2,0.205519,0.2,0.200296,0.200296,0.2
3,0.2,0.2,0.289038,0.203660,0.378297,0.501634,0.368600,0.211779,0.208683,0.200483,...,0.2,0.2,0.200715,0.201650,0.2,0.204649,0.2,0.200400,0.200400,0.2


### Model Naive Bayes With LDA

Kode tersebut melibatkan evaluasi model Naive Bayes yang telah dilatih menggunakan representasi dokumen dalam ruang topik dari model Latent Dirichlet Allocation (LDA). Proses evaluasi ini mencakup langkah-langkah berikut:

1. **Mengambil Model Naive Bayes Terbaik:**
   - Mengambil model Naive Bayes terbaik yang telah diidentifikasi selama eksperimen. Model ini disimpan dalam `best_param`.

2. **Melakukan Prediksi pada Data Pengujian:**
   - Menggunakan model Naive Bayes terbaik untuk melakukan prediksi pada data pengujian. Representasi dokumen dalam ruang topik diperoleh dari model LDA.

3. **Menghitung Akurasi:**
   - Menghitung akurasi model, yaitu sejauh mana prediksi model sesuai dengan label sebenarnya pada data pengujian.

4. **Menampilkan Laporan Klasifikasi:**
   - Mencetak laporan klasifikasi yang menyajikan metrik seperti presisi, recall, dan f1-score untuk setiap kelas. Laporan ini memberikan informasi rinci tentang kinerja model pada setiap kelas.

5. **Menampilkan Matriks Kebingungan:**
   - Mencetak matriks kebingungan yang menunjukkan seberapa baik model dapat memprediksi setiap kelas pada data pengujian. Matriks ini berguna untuk mengevaluasi kinerja model lebih mendalam.

Proses ini memberikan gambaran komprehensif tentang seberapa baik model Naive Bayes yang telah dilatih dapat mengklasifikasikan dokumen berdasarkan representasi dalam ruang topik dari model LDA. Evaluasi ini memungkinkan untuk menilai kinerja model dengan lebih baik dan memahami sejauh mana model mampu mengenali kategori dokumen dengan tepat.

In [19]:
# Membuat model Naive Bayes
model = best_param['model']

# Melakukan prediksi pada data pengujian
y_pred = model.predict(lda_x_test)

# Menghitung akurasi
accuracy = accuracy_score(y_test, y_pred)
print("Akurasi:", accuracy)

# Menampilkan laporan klasifikasi
print("Laporan Klasifikasi:")
print(classification_report(y_test, y_pred))

# Menampilkan matriks kebingungan
confusion = confusion_matrix(y_test, y_pred)
print("Confusion Matriks:")
print(confusion)

Akurasi: 0.7288888888888889
Laporan Klasifikasi:
              precision    recall  f1-score   support

     Ekonomi       0.66      0.33      0.44       141
    Olahraga       0.93      0.93      0.93       161
   Teknologi       0.60      0.89      0.72       148

    accuracy                           0.73       450
   macro avg       0.73      0.72      0.70       450
weighted avg       0.74      0.73      0.71       450

Confusion Matriks:
[[ 47  10  84]
 [  9 150   2]
 [ 15   2 131]]


### Predict

In [21]:
data = ['Jakarta, CNN Indonesia -- Isu negatif, hoaks, hingga ujaran kebencian mengenai pengungsi Rohingya di Indonesia merebak di sejumlah platform media sosial. Siapa yang pertama kali menyebar narasi kebencian soal Rohingya di Aceh? Merujuk laporan yang didapat CNNIndonesia.com dari salah satu sumber PBB, narasi kebencian mengenai pengungsi Rohingya sudah dimulai sejak 21 November. Total ada empat unggahan dari akun Instagram @UNinIndonesia mendapat serangan komentar kebencian mengenai Rohingya. "Bahkan unggahan lain (non-Rohingya) di akun @UNinIndonesia juga mendapat komentar negatif terkait Rohingya, karena unggahan sebelumnya dikolaborasikan dengan @UNHCRIndonesia dan @UNHCRIndonesia telah menonaktifkan komentar," tulis laporan itu, dikutip Selasa (12/12). Menurut laporan itu, ada 17.380 komentar mengenai Rohingya dan 91 persen di antaranya adalah komentar kebencian. Laporan PBB tersebut juga menunjukkan frekuensi penggunaan kata oleh para warganet yang berkomentar. Hampir semua komentar melontarkan komentar yang sama, yaitu "bubarkan" UNHCR/PBB, tapi dengan berbagai variasi sinonimnya, seperti "tutup". Tidak hanya itu, yang paling unik adalah tingginya frekuensi penggunaan kata penghubung slang seperti "aja" dan "buat", yang sangat jarang tercatat sebagai kata kunci yang sering muncul dalam analisis media sosial. "Hal ini menunjukkan bahwa pesan-pesan tersebut telah diatur dan bukan berasal dari akar rumput. Banyak pesan yang kontra terhadap UNHCR, bukan hanya terhadap etnis Rohingya saja," lanjutnya. Laporan PBB itu juga mengungkap bahwa jangkauan atau reach sebagian besar unggahan mengenai pengungsi Rohingya 95 persen bukan followers akun @UNinIndonesia. Kemudian, sejumlah akun berkomentar bahwa mereka tidak mengetahui apa itu UNHCR, tapi memberikan narasi kebencian. Tidak hanya itu, jangkauan postingan feed ke non-followers rata-rata adalah 59 ribu per postingan, namun dengan jumlah share rata-rata hanya 177 per postingan, sementara rasio normal untuk share:reach adalah 1:10. Hal ini menunjukkan komentator-komentar tersebut mungkin telah menerima tautan langsung ke postingan PBB, memberikan komentar negatif yang anti-pengungsi dan mendesak "menutup UNHCR". Laporan itu juga menemukan satu akun yang paling vokal dalam isu negatif mengenai etnis Rohingya, yakni @hamzali_abradinezad di Instagram dan @allinezad di Tiktok. PBB mengklaim sudah melaporkan akun ini ke pihak Meta dan perwakilan TikTok di Indonesia. "Satu akun tampaknya menjadi inti dari kampanye ini; karena "ujaran kebencian" tidak dapat ditujukan kepada sebuah organisasi, hanya kepada orang-orang, kami telah melaporkannya atas pernyataannya yang mengatakan bahwa Rohingya harus diusir karena Indonesia akan menjadi Palestina berikutnya setelah kedatangan mereka," tulis PBB dalam laporan tersebut.']
a = tfidf_vectorizer.transform(data).toarray()
b = lda.transform(a)
model.predict(b)

array(['Teknologi'], dtype='<U9')

### Save Model

In [22]:
joblib.dump(lda, "lda.pkl")
joblib.dump(model, "naive bayes.pkl")
joblib.dump(model, "naive bayes.pkl")

['naive bayes.pkl']

### Save Vectorizer

In [23]:
joblib.dump(tfidf_vectorizer, 'vectorizer.pkl')

['vectorizer.pkl']